# Задание 2. SQL

In [9]:
import pandahouse as ph

Подключение к базе данных

In [10]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

### 2.1 Очень усердные ученики.

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [11]:
query_1 = '''
    SELECT
        COUNT(st_id) AS count
    FROM
        (SELECT
            st_id
        FROM 
            peas
        GROUP BY
            st_id
        HAVING
            SUM(correct) >= 20)
    '''

In [16]:
task_1 = ph.read_clickhouse(query=query_1, connection=connection_default)
task_1

,count
0,136


### 2.2 Оптимизация воронки

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.
    
Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU

- ARPAU 

- CR в покупку 

- СR активного пользователя в покупку 

- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

In [14]:
query_2 = '''
    WITH revenue_table AS (
        SELECT 
            SUM(money) FILTER(WHERE test_grp = 'pilot') AS sum_pilot,
            SUM(money) FILTER(WHERE test_grp = 'control') AS sum_control
        FROM
            final_project_check AS l
        LEFT JOIN studs AS r
        ON l.st_id = r.st_id
    ),

    count_all_users AS (
        SELECT
            COUNT(st_id) FILTER(WHERE test_grp = 'pilot') AS count_pilot_users,
            COUNT(st_id) FILTER(WHERE test_grp = 'control') AS count_control_users
        FROM
            studs
    ),

    active_users AS (
        SELECT
            st_id,
            test_grp
        FROM
            (SELECT
                st_id
            FROM 
                peas
            GROUP BY
                st_id
            HAVING
                SUM(correct) > 10) AS t1
        JOIN studs
        ON t1.st_id = studs.st_id
    ),

    count_active_users AS (
        SELECT
            COUNT(st_id) FILTER(WHERE test_grp = 'pilot') AS count_active_pilot_users,
            COUNT(st_id) FILTER(WHERE test_grp = 'control') AS count_active_control_users
        FROM
            active_users
    ),

    count_users_purchased AS (
        SELECT
            COUNT(st_id) FILTER(WHERE test_grp = 'pilot') AS count_users_purchased_pilot,
            COUNT(st_id) FILTER(WHERE test_grp = 'control') AS count_users_purchased_control
        FROM    
            (SELECT 
                DISTINCT st_id,
                test_grp
            FROM
                final_project_check AS l
            LEFT JOIN studs AS r
            ON l.st_id = r.st_id)
    ),

    count_active_users_purchased AS (
        SELECT
            COUNT(st_id) FILTER(WHERE test_grp = 'pilot') AS count_active_users_purchased_pilot,
            COUNT(st_id) FILTER(WHERE test_grp = 'control') AS count_active_users_purchased_control
        FROM    
            (SELECT
                DISTINCT st_id,
                test_grp
            FROM
                final_project_check AS l
            JOIN studs AS r
            ON l.st_id = r.st_id
            WHERE
                st_id IN (SELECT st_id FROM active_users))
    ),

    active_users_by_math AS (
        SELECT
            st_id
        FROM
            peas
        WHERE
            subject = 'Math'
        GROUP BY
            st_id
        HAVING
            SUM(correct) >= 2
    ),

    count_active_users_by_math AS (
        SELECT
            COUNT(st_id) FILTER(WHERE test_grp = 'pilot') AS count_active_users_math_pilot,
            COUNT(st_id) FILTER(WHERE test_grp = 'control') AS count_active_users_math_control
        FROM
            active_users_by_math AS l
        JOIN studs AS r
        ON l.st_id = r.st_id
    ),

    count_active_by_math_purchased AS (
        SELECT
            COUNT(st_id) FILTER(WHERE test_grp = 'pilot') AS count_users_purchased_math_pilot,
            COUNT(st_id) FILTER(WHERE test_grp = 'control') AS count_users_purchased_math_control
        FROM
            (SELECT
                st_id   
            FROM
                final_project_check
            WHERE
                subject = 'Math' AND st_id IN (SELECT * FROM active_users_by_math)) AS l
        JOIN studs AS r
        ON l.st_id = r.st_id
    )

    SELECT
        (SELECT sum_pilot FROM revenue_table) / (SELECT count_pilot_users FROM count_all_users) AS arpu_pilot,
        (SELECT sum_control FROM revenue_table) / (SELECT count_control_users FROM count_all_users) AS arpu_control,
        (SELECT sum_pilot FROM revenue_table) / (SELECT count_active_pilot_users FROM count_active_users) AS arpau_pilot,
        (SELECT sum_control FROM revenue_table) / (SELECT count_active_control_users FROM count_active_users) AS arpau_control,
        (SELECT count_users_purchased_pilot FROM count_users_purchased) / (SELECT count_pilot_users FROM count_all_users) AS CR_pilot,
        (SELECT count_users_purchased_control FROM count_users_purchased) / (SELECT count_control_users FROM count_all_users) AS CR_control,
        (SELECT count_active_users_purchased_pilot FROM count_active_users_purchased) / (SELECT count_active_pilot_users FROM count_active_users) AS CR_active_pilot,
        (SELECT count_active_users_purchased_control FROM count_active_users_purchased) / (SELECT count_active_control_users FROM count_active_users) AS CR_active_control,
        (SELECT count_users_purchased_math_pilot FROM count_active_by_math_purchased) / (SELECT count_active_users_math_pilot FROM count_active_users_by_math) AS CR_active_math_pilot,
        (SELECT count_users_purchased_math_control FROM count_active_by_math_purchased) / (SELECT count_active_users_math_control FROM count_active_users_by_math) AS CR_active_math_control
    FROM
        studs
    LIMIT 1
    ''' 

In [15]:
task_2 = ph.read_clickhouse(query=query_2, connection=connection_default)
task_2

,arpu_pilot,arpu_control,arpau_pilot,arpau_control,CR_pilot,CR_control,CR_active_pilot,CR_active_control,CR_active_math_pilot,CR_active_math_control
0,11508.474576,4540.983607,35364.583333,10905.511811,0.108475,0.04918,0.260417,0.110236,0.095238,0.061224
